Load graphml file and transform into h5 file using pandana

In [24]:
import networkx as nx

G_temp=nx.read_graphml("./networks/gipuzkoa_walk_with_geometry.graphml")

In [25]:
import pandas as pd

node_df=pd.DataFrame.from_dict(dict(G_temp.nodes(data=True)), orient="index")
edge_df=nx.to_pandas_edgelist(G_temp)

In [26]:
print(node_df.head())

                 y           x street_count highway  ref
448001  43.1795625  -2.4899828            3     NaN  NaN
448006  43.1826552  -2.4814224            4     NaN  NaN
451277  43.2049133   -2.425101            3     NaN  NaN
451289  43.1927813  -2.4381722            3     NaN  NaN
469501  43.1911116  -2.4516244            3     NaN  NaN


In [27]:
# rename length to distance
edge_df=edge_df.rename(columns={"length": "distance"})
print(edge_df.head())

   source      target                                           geometry  \
0  448001  3934729820  LINESTRING (-2.4899828 43.1795625, -2.489874 4...   
1  448001    25438648  LINESTRING (-2.4899828 43.1795625, -2.4900901 ...   
2  448001      916343  LINESTRING (-2.4899828 43.1795625, -2.4901355 ...   
3  448006  5949227426  LINESTRING (-2.4814224 43.1826552, -2.4816299 ...   
4  448006     6928916  LINESTRING (-2.4814224 43.1826552, -2.4814342 ...   

  reversed      ref tunnel      highway oneway maxspeed service junction  \
0    False      NaN    NaN     tertiary  False      NaN     NaN      NaN   
1     True      NaN    NaN  residential  False      NaN     NaN      NaN   
2     True  BI-3344    NaN     tertiary  False      NaN     NaN      NaN   
3     True      NaN    NaN     tertiary  False      NaN     NaN      NaN   
4     True      NaN    NaN  residential  False      NaN     NaN      NaN   

  distance bridge landuse                   name access lanes  \
0  113.604    NaN    

In [28]:
# Convert columns to numeric, forcing errors='coerce' to handle non-numeric values
edge_df["distance"] = pd.to_numeric(edge_df["distance"], errors="coerce")

In [29]:
# distance miles, convert distance from meters to miles
edge_df["distance_miles"] = edge_df["distance"] / 1609.34

In [30]:
# calculate travel time in seconds, assuming 5 km/h and distance in meters
edge_df["travel_time"] = edge_df["distance"] / 5000 * 3600
print(edge_df.head())

   source      target                                           geometry  \
0  448001  3934729820  LINESTRING (-2.4899828 43.1795625, -2.489874 4...   
1  448001    25438648  LINESTRING (-2.4899828 43.1795625, -2.4900901 ...   
2  448001      916343  LINESTRING (-2.4899828 43.1795625, -2.4901355 ...   
3  448006  5949227426  LINESTRING (-2.4814224 43.1826552, -2.4816299 ...   
4  448006     6928916  LINESTRING (-2.4814224 43.1826552, -2.4814342 ...   

  reversed      ref tunnel      highway oneway maxspeed service  ... distance  \
0    False      NaN    NaN     tertiary  False      NaN     NaN  ...  113.604   
1     True      NaN    NaN  residential  False      NaN     NaN  ...  139.660   
2     True  BI-3344    NaN     tertiary  False      NaN     NaN  ...   42.152   
3     True      NaN    NaN     tertiary  False      NaN     NaN  ...   20.781   
4     True      NaN    NaN  residential  False      NaN     NaN  ...   78.047   

   bridge landuse                   name access lanes  \

In [31]:
# just keep index, x and y from node_df
node_df=node_df[["x", "y"]]

# just keep source, target and distance from edge_df
edge_df=edge_df[["source", "target", "distance_miles"]]

In [35]:
# rename distance_miles to distance
edge_df=edge_df.rename(columns={"distance_miles": "distance"})

In [36]:
print(node_df.head())
print(edge_df.head())

                 x           y
448001  -2.4899828  43.1795625
448006  -2.4814224  43.1826552
451277   -2.425101  43.2049133
451289  -2.4381722  43.1927813
469501  -2.4516244  43.1911116
   source      target  distance
0  448001  3934729820  0.070590
1  448001    25438648  0.086781
2  448001      916343  0.026192
3  448006  5949227426  0.012913
4  448006     6928916  0.048496


In [37]:
import pandana as pdna

node_x=node_df["x"]
node_y=node_df["y"]
edge_from=edge_df["source"]
edge_to=edge_df["target"]
edge_weights=pd.DataFrame(edge_df["distance"])

walk_network = pdna.Network(node_x, node_y, edge_from, edge_to, edge_weights, twoway=True)

print(walk_network)

In [38]:
network_file="./networks/walk_network_miles.h5"
walk_network.save_hdf5(network_file)